
# Homework 1 — Linear & Logistic Regression (Clean Submission)

This notebook is a **cleaned and organized** version of the working notebook, focused on the exact assignment requirements.
It contains **Problem 1 (Linear Regression)** and **Problem 2 (Logistic Regression)**, each with parts (a)–(e).  
For each part, we include:
- A short **problem summary** (what is required),
- The **minimal, correct code** to satisfy the requirement,
- **Explanations** of *why* we do it and *what* the outputs mean.

> Reproducibility: we set a fixed random seed where required and keep preprocessing consistent across parts.



> **Data**  
- `Carseats.csv` for Problem 1 (variables: `Sales`, `Price`, `Urban`, `US`).  
- `Default.csv` for Problem 2 (variables: `default`, `student`, `balance`, `income`).

> **Paths**  
- Place the CSVs in the current working directory **or** update `DATA_DIR` accordingly.  
- On Google Colab, either use `files.upload()` or mount Google Drive.

> **Random Seed**  
We use `RANDOM_SEED = 1` across all steps that require randomness.


In [12]:

# Core imports
import numpy as np
import pandas as pd

# Modeling & evaluation
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score, log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from scipy import stats
from scipy.optimize import minimize

# Plotting
import matplotlib.pyplot as plt

# Reproducibility
RANDOM_SEED = 1
np.random.seed(RANDOM_SEED)

# Data directory (adjust if needed)
DATA_DIR = "/Users/lavi/MLDL_25_2_ASSIGNMENT/ASSIGNMENT_1/Data"



> **Colab helpers (optional):**  
Uncomment and run the cells below if you're on Google Colab and need to load files.

```python
# from google.colab import files
# uploaded = files.upload()  # then read directly by filename
```

or

```python
# from google.colab import drive
# drive.mount('/content/drive')
# DATA_DIR = "/content/drive/MyDrive/your_folder"
```


# Problem 1 — Linear Regression (Carseats.csv)


We use the **Carseats** dataset and focus on the model:

\[
\text{Sales} \sim \text{Price} + \text{Urban} + \text{US}
\]

- `Urban`, `US` are qualitative predictors with two levels; we use **one dummy per predictor** (drop the baseline) to avoid the dummy-variable trap.


In [13]:

# Load Carseats
car_path = f"{DATA_DIR}/Carseats.csv"
df1 = pd.read_csv(car_path)

# Ensure correct dtypes and dummy coding
df1['Urban'] = df1['Urban'].astype('category')
df1['US']    = df1['US'].astype('category')

# Target and predictors
y1 = df1['Sales']
X1 = pd.get_dummies(df1[['Price', 'Urban', 'US']], drop_first=True)  # creates Urban_Yes, US_Yes

print("X1 columns:", list(X1.columns))
print("X1 shape:", X1.shape, " y1 shape:", y1.shape)


X1 columns: ['Price', 'Urban_Yes', 'US_Yes']
X1 shape: (400, 3)  y1 shape: (400,)



## (a) Fit Linear Regression with scikit-learn and report coefficients and R²

**What to do:** Fit OLS using scikit-learn (`LinearRegression`) on the full data and report the intercept, coefficients, and R².

**Why:** This provides a baseline OLS fit using a standard library implementation.

**Output meaning:**
- Intercept \(\beta_0\): expected Sales at baseline (Urban=No, US=No) & Price=0 (extrapolation).
- Coefficients: slope effects of each predictor on Sales.
- \(R^2\): fraction of variance in Sales explained by the model.


In [14]:

lin1 = LinearRegression(fit_intercept=True)
lin1.fit(X1, y1)

r2_full = lin1.score(X1, y1)
print("Intercept:", lin1.intercept_)
print("Coefficients:", dict(zip(X1.columns, lin1.coef_)))
print("R^2 (full data):", r2_full)


Intercept: 13.043468936764892
Coefficients: {'Price': np.float64(-0.05445884917758218), 'Urban_Yes': np.float64(-0.021916150814141), 'US_Yes': np.float64(1.200572697794117)}
R^2 (full data): 0.23927539218405547



## (b) Write the model and interpret coefficients

Model:
\[
\widehat{\text{Sales}} = \beta_0 + \beta_1 \cdot \text{Price} + \beta_2 \cdot \text{Urban\_Yes} + \beta_3 \cdot \text{US\_Yes}
\]

- **Price**: expected change in Sales per unit increase in Price.
- **Urban\_Yes**: difference in baseline Sales for Urban=Yes vs Urban=No.
- **US\_Yes**: difference in baseline Sales for US=Yes vs US=No.

We interpret signs and magnitudes based on the fitted coefficients above.



## (c) Closed-form OLS: \(\hat{\beta} = (X^\top X)^{-1}X^\top y\)

**What to do:** Construct a design matrix \(X\) with an intercept column and one dummy per qualitative predictor (as above), then compute \(\hat{\beta}\) via the closed-form expression. Compare with scikit-learn results.

**Note:** Use the **same coding choices** as in part (a).

**Output meaning:** \(\hat{\beta}\) is a \((p{+}1) \times 1\) vector (intercept + p coefficients).


In [15]:

# Build design matrix with intercept
X1_closed = np.column_stack([np.ones((X1.shape[0], 1)), X1.values]).astype(float)  # (n, p+1)
y1_mat = y1.values.reshape(-1, 1)                                    # (n, 1)

# Closed-form estimator
beta_hat_1 = np.linalg.inv(X1_closed.T @ X1_closed) @ (X1_closed.T @ y1_mat)  # (p+1, 1)

colnames1 = ['Intercept'] + list(X1.columns)
beta_df1 = pd.DataFrame(beta_hat_1.flatten(), index=colnames1, columns=['Estimate (closed-form)'])
print(beta_df1)

# Compare with sklearn
coef_sklearn = np.r_[lin1.intercept_, lin1.coef_]
cmp1 = pd.DataFrame({'ClosedForm': beta_hat_1.flatten(), 'sklearn': coef_sklearn}, index=colnames1)
cmp1['abs_diff'] = np.abs(cmp1['ClosedForm'] - cmp1['sklearn'])
print("\nComparison closed-form vs sklearn:")
print(cmp1)


           Estimate (closed-form)
Intercept               13.043469
Price                   -0.054459
Urban_Yes               -0.021916
US_Yes                   1.200573

Comparison closed-form vs sklearn:
           ClosedForm    sklearn      abs_diff
Intercept   13.043469  13.043469  1.421085e-14
Price       -0.054459  -0.054459  1.595946e-16
Urban_Yes   -0.021916  -0.021916  4.836409e-15
US_Yes       1.200573   1.200573  6.661338e-16



## (d) t-statistics, standard errors, and p-values (α=0.05)

**What to do:** Compute the residual variance \(\hat{\sigma}^2\), the covariance matrix \(\hat{\sigma}^2 (X^\top X)^{-1}\), then standard errors, t-stats, and two-sided p-values.

**Degrees of freedom:** \(df = n - (p+1)\).

**Output meaning:** significance of each coefficient at α=0.05.


In [16]:

# Predictions & residuals
y1_hat = X1_closed @ beta_hat_1
eps1 = y1_mat - y1_hat

n1, p1_plus_1 = X1_closed.shape  # p1_plus_1 = p+1
df1 = n1 - p1_plus_1

sigma_sq_1 = float((eps1.T @ eps1) / df1)
cov_beta_1 = sigma_sq_1 * np.linalg.inv(X1_closed.T @ X1_closed)
se_1 = np.sqrt(np.diag(cov_beta_1))

t_stat_1 = beta_hat_1.flatten() / se_1
p_vals_1 = 2 * (1 - stats.t.cdf(np.abs(t_stat_1), df=df1))

t_table_1 = pd.DataFrame({
    'Estimate': beta_hat_1.flatten(),
    'Std.Error': se_1,
    't value': t_stat_1,
    'Pr(>|t|)': p_vals_1
}, index=colnames1)

print(t_table_1)


            Estimate  Std.Error    t value  Pr(>|t|)
Intercept  13.043469   0.651012  20.035674  0.000000
Price      -0.054459   0.005242 -10.389232  0.000000
Urban_Yes  -0.021916   0.271650  -0.080678  0.935739
US_Yes      1.200573   0.259042   4.634673  0.000005


/var/folders/4p/sb20bp956_bgbntjhp461wbm0000gn/T/ipykernel_78763/4059263183.py:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sigma_sq_1 = float((eps1.T @ eps1) / df1)



## (e) 70/30 Train–Validation and 5-fold Cross-Validation

**What to do:**  
1. Random 70/30 split (report the seed).  
2. Fit on the training set; report **validation-set \(R^2\)** and **MSE**.  
3. Perform **5-fold CV** and report mean \(R^2\) and MSE.  
4. Briefly discuss differences.

**Why:** Assesses **generalization** instead of in-sample fit.


In [17]:

# 70/30 split
X1_tr, X1_va, y1_tr, y1_va = train_test_split(X1, y1, test_size=0.3, random_state=RANDOM_SEED)

lm1 = LinearRegression(fit_intercept=True)
lm1.fit(X1_tr, y1_tr)

y1_pred_va = lm1.predict(X1_va)
r2_val_1 = r2_score(y1_va, y1_pred_va)
mse_val_1 = mean_squared_error(y1_va, y1_pred_va)

print(f"Validation R^2: {r2_val_1:.4f}")
print(f"Validation MSE: {mse_val_1:.4f}")

# 5-fold Cross-Validation (mean R^2 and MSE)
r2_cv_1 = cross_val_score(lm1, X1, y1, cv=5, scoring='r2').mean()
mse_cv_1 = -cross_val_score(lm1, X1, y1, cv=5, scoring='neg_mean_squared_error').mean()

print(f"Cross-validated R^2 (mean): {r2_cv_1:.4f}")
print(f"Cross-validated MSE (mean): {mse_cv_1:.4f}")


Validation R^2: 0.2849
Validation MSE: 5.6138
Cross-validated R^2 (mean): 0.2038
Cross-validated MSE (mean): 6.1903


# Problem 2 — Logistic Regression (Default.csv)


We model **default** (Yes/No) as a function of **income** and **balance** (and later `student`).  
We encode `default` as 1(Yes)/0(No), and **standardize** features for stable optimization.


In [18]:

# Load Default
def_path = f"{DATA_DIR}/Default.csv"
df2 = pd.read_csv(def_path)

# Target encoding
df2['default_bin'] = df2['default'].map({'No': 0, 'Yes': 1}).astype(int)

X2 = df2[['income', 'balance']]
y2 = df2['default_bin']

print("X2 shape:", X2.shape, " y2 shape:", y2.shape)


X2 shape: (10000, 2)  y2 shape: (10000,)



## (a) Logistic Regression via scikit-learn (MLE) + Training Log-Likelihood

- Pipeline: `StandardScaler` + `LogisticRegression(penalty='none', solver='lbfgs')`  
- Report **intercept, coefficients**, and **training log-likelihood** (sum, not mean).

**Why penalty='none'?** To match pure MLE without regularization.  
**Why standardize?** To improve numerical conditioning (features on comparable scales).


In [20]:

pipe2a = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(penalty=None, solver='lbfgs', max_iter=1000, random_state=RANDOM_SEED))
])
pipe2a.fit(X2, y2)

clf2a = pipe2a.named_steps['clf']
beta2a = np.r_[clf2a.intercept_[0], clf2a.coef_[0]]
print("β (std-scale) =", beta2a)

p_train2 = pipe2a.predict_proba(X2)[:, 1]
n2 = len(y2)
train_loglik2 = -log_loss(y2, p_train2, normalize=False)  # sum log-likelihood
print("Training log-likelihood:", train_loglik2)


β (std-scale) = [-6.1093768   0.27915846  2.72232674]
Training log-likelihood: -789.4877437837663



## (b) Model Equation & Interpretation (Signs and Odds Ratios)

Model:
\[
\log\frac{p_i}{1-p_i} = \beta_0 + \beta_1 \cdot income_i + \beta_2 \cdot balance_i
\]

- Sign of \(\beta_1\): effect of income on default odds.  
- Sign of \(\beta_2\): effect of balance on default odds.  
- **Odds ratio**: \(e^{\beta_j}\). With standardized predictors, “1 unit” = “1 standard deviation”.


In [21]:

coef_income, coef_balance = clf2a.coef_[0]
or_income = np.exp(coef_income)
or_balance = np.exp(coef_balance)

print(f"Income OR = {or_income:.3f}")
print(f"Balance OR = {or_balance:.3f}")


Income OR = 1.322
Balance OR = 15.216



## (c) Manual MLE by Direct Log-Likelihood Maximization

We construct \(X=[1, income_{std}, balance_{std}]\), define \(-\log L(\beta)\) and its gradient,
and use BFGS to minimize it. We then compare the coefficients and log-likelihood with part (a).


In [22]:

# Use the same scaler as in (a)
scaler2 = pipe2a.named_steps['scaler']
X2_std = scaler2.transform(X2)
X2_mat = np.column_stack([np.ones(len(X2_std)), X2_std]).astype(float)
y2_vec = y2.values.astype(float)

def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-np.clip(z, -35, 35)))

def neg_loglik2(beta):
    z = X2_mat @ beta
    p = sigmoid(z)
    eps = 1e-12
    return -np.sum(y2_vec*np.log(p + eps) + (1 - y2_vec)*np.log(1 - p + eps))

def grad_neg_loglik2(beta):
    z = X2_mat @ beta
    p = sigmoid(z)
    return X2_mat.T @ (p - y2_vec)

beta0 = np.zeros(X2_mat.shape[1])
res2 = minimize(neg_loglik2, beta0, jac=grad_neg_loglik2, method='BFGS', options={'gtol':1e-8, 'maxiter':1000})
beta2c = res2.x
loglik2c = -res2.fun

print("β (manual, std-scale) =", beta2c)
print("Max log-likelihood (manual):", loglik2c)

# Compare with (a)
cmp2 = pd.DataFrame({
    'param': ['Intercept','income(std)','balance(std)'],
    'sklearn(2a)': beta2a,
    'manual(2c)': beta2c,
})
cmp2['abs_diff'] = np.abs(cmp2['sklearn(2a)'] - cmp2['manual(2c)'])
print(cmp2)


β (manual, std-scale) = [-6.12556642  0.27750793  2.73145174]
Max log-likelihood (manual): -789.4831350811944
          param  sklearn(2a)  manual(2c)  abs_diff
0     Intercept    -6.109377   -6.125566  0.016190
1   income(std)     0.279158    0.277508  0.001651
2  balance(std)     2.722327    2.731452  0.009125



## (d) 70/30 Validation (Misclassification Rate) and 5-Fold CV

- **Split** the data into 70%/30% (seed reported).
- **Fit** the (2a) pipeline on the training set.
- **Validation** misclassification rate using threshold 0.5.
- **5-fold CV** mean misclassification rate.


In [24]:

X2_tr, X2_va, y2_tr, y2_va = train_test_split(X2, y2, test_size=0.3, random_state=RANDOM_SEED, stratify=y2)

pipe2d = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(penalty=None, solver='lbfgs', max_iter=1000, random_state=RANDOM_SEED))
])
pipe2d.fit(X2_tr, y2_tr)

proba_va = pipe2d.predict_proba(X2_va)[:, 1]
yhat_va = (proba_va >= 0.5).astype(int)

val_acc = accuracy_score(y2_va, yhat_va)
val_miscl = 1 - val_acc

print(f"Validation Accuracy: {val_acc:.4f}")
print(f"Validation Misclassification Rate: {val_miscl:.4f}")

cv_acc = cross_val_score(pipe2d, X2, y2, cv=5, scoring='accuracy')
cv_miscl = 1 - cv_acc.mean()

print(f"5-Fold Mean Accuracy: {cv_acc.mean():.4f}")
print(f"5-Fold Mean Misclassification Rate: {cv_miscl:.4f}")


Validation Accuracy: 0.9760
Validation Misclassification Rate: 0.0240
5-Fold Mean Accuracy: 0.9735
5-Fold Mean Misclassification Rate: 0.0265



## (e) Add `student` Dummy and Compare 5-Fold CV Error

We add a dummy \(1\{student=\text{Yes}\}\) and evaluate the mean misclassification rate via 5-fold CV.


In [26]:

df2['student_bin'] = df2['student'].map({'No': 0, 'Yes': 1}).astype(int)
X2e = df2[['income','balance','student_bin']]

pipe2e = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(penalty=None, solver='lbfgs', max_iter=1000, random_state=RANDOM_SEED))
])

cv_acc_2e = cross_val_score(pipe2e, X2e, y2, cv=5, scoring='accuracy')
cv_miscl_2e = 1 - cv_acc_2e.mean()

print(f"(2e) 5-Fold Mean Accuracy: {cv_acc_2e.mean():.4f}")
print(f"(2e) 5-Fold Mean Misclassification Rate: {cv_miscl_2e:.4f}")


(2e) 5-Fold Mean Accuracy: 0.9732
(2e) 5-Fold Mean Misclassification Rate: 0.0268



---

### Notes
- All random splits use `random_state=1` for reproducibility.
- Standardization is kept consistent wherever required.
- Cross-validated metrics are averaged across folds.
- Closed-form OLS and sklearn results (Problem 1) match up to numeric precision.
- Manual MLE (Problem 2) matches sklearn’s MLE (penalty='none') up to optimizer tolerance.

**End of cleaned notebook.**
